In [ ]:
%load_ext autoreload
%autoreload 2
import jax
from jax import random
from environments import RodentSingleClipTrack
import numpy as np
import mediapy as media

params = {
    "scale_factor": .9,
    "solver": "cg",
    "iterations": 6,
    "ls_iterations": 3,
    "clip_path": "12_21_1_250.h5",
    "end_eff_names": [
        "foot_L",
        "foot_R",
        "hand_L",
        "hand_R",
    ],
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
env = RodentSingleClipTrack(params)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:344: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  link = jax.tree_map(lambda x: x[1:].copy(), link)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:374: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  motion = jax.tree_map(lambda *x: np.concatenate(x), *motions)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/brax/io/mjcf.py:378: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  limit = jax.tree_map(lambda *x: np.concatenate(x), *limits)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-M

In [10]:
type(env._ref_traj.quaternion)

numpy.ndarray

In [11]:
env.body_idxs

Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17], dtype=int32)

In [48]:
env._ref_traj.joints_velocity.shape

(67, 250)

In [38]:
rng = random.PRNGKey(0)
state = env.reset(rng)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx

In [14]:
state.info['start_frame']

Array(56, dtype=int32)

In [15]:
env._ref_traj.velocity[:, state.info['start_frame']]

array([ 0.40821564, -0.20567179, -0.11344627])

In [52]:
state.pipeline_state.subtree_com[1].shape

(3,)

In [53]:
env._ref_traj.center_of_mass.shape

(3, 250)

In [59]:
env._ref_traj

ReferenceClip(angular_velocity=array([[-3.04612576e-01,  8.02327207e+00,  4.89195633e+00,
        -8.00826400e+00,  3.80873255e+00, -3.09084014e+00,
         1.27929038e+00,  2.53808568e-02,  2.73956385e+00,
        -1.15680768e+00, -3.14908129e+00, -9.01270022e-01,
        -1.46512429e+00, -4.04753144e-01,  2.65951957e+00,
         4.90195616e+00,  6.07379245e-01, -2.42692911e+00,
         1.09252837e+00,  2.28267820e-01, -1.45807625e+00,
        -5.39465846e-01,  1.65175688e-02,  1.06735046e-01,
         1.40939529e+00,  2.35438178e+00,  7.87581096e-01,
        -5.77657216e-01, -1.28614850e+00, -2.45397146e+00,
         2.15833431e-01, -1.25466836e-01, -1.62547778e-01,
         1.73505884e-01, -9.03798077e-02, -8.97738321e-01,
        -1.04056377e+00, -1.32957752e-01, -2.46977140e-01,
         6.36331975e-01,  1.90574762e+00,  4.19152679e-01,
         2.79136408e+00,  3.32308625e+00,  2.32628282e+00,
         2.00793650e+00, -7.86782568e-01, -2.48604899e+00,
        -2.31341238e+00, 

In [42]:
rollout = [state.pipeline_state]

for i in (range(2)):
    ctrl = jax.numpy.array(np.random.uniform(-1,1, env.sys.nu))
    state = env.step(state, ctrl)
    rollout.append(state.pipeline_state)

/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx/_src/scan.py:65: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  return jax.tree_map(take, obj)
/Users/kevinb/Desktop/salk/rodent/VNL-Brax-Imitation-Main/brax-imitation/lib/python3.10/site-packages/mujoco/mjx

TypeError: sub got incompatible shapes for broadcasting: (73,), (67,).

In [ ]:
media.show_video(env.render(rollout,camera='close_profile'), fps=1.0 / env.dt)